# Pinecone Vector Store - Hybrid Search

#### Creating a Pinecone Index

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import pinecone

In [ ]:
api_key = ""
pinecone.init(api_key=api_key, environment="us-west1-gcp")

In [ ]:
pinecone.describe_index("quickstart")

In [ ]:
# dimensions are for text-embedding-ada-002
# NOTE: needs dotproduct for hybrid search
pinecone.create_index("quickstart", dimension=1536, metric="dotproduct", pod_type="p1")

In [ ]:
pinecone_index = pinecone.Index("quickstart")

#### Load documents, build the PineconeVectorStore

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import PineconeVectorStore
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader("../paul_graham_essay/data").load_data()

In [ ]:
# set add_sparse_vector=True to compute sparse vectors during upsert
from llama_index.storage.storage_context import StorageContext


vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(vector_store_query_mode="hybrid")
response = query_engine.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))